In [7]:
import importlib
import numpy as np
from openai import OpenAI
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

### Step 1: Prep Data

In [8]:
sp = pd.read_pickle('data/sp500.pkl')
fomc = pd.read_pickle('data/fomc_statements.pkl')
fomc = fomc.drop(columns='statement')
fomc = fomc.rename(columns={'cleaned_statement':'statement'})
fomc = fomc[fomc['date']>='2000-01-03'].reset_index(drop=True)
sp['ret'] = np.log(1 + sp['ret'])

In [9]:
holding_periods = [1, 5, 10]

In [10]:
# percent stocks in long/short
portfolio_size = 0.1

top_percentile = 1 - portfolio_size
bottom_percentile = portfolio_size

In [11]:
fomc

,date,statement
0,2000-02-02,immediate release federal open market committe...
1,2000-03-21,immediate release federal open market committe...
2,2000-05-16,immediate release federal open market committe...
3,2000-06-28,immediate release federal open market committe...
4,2000-08-22,immediate release federal open market committe...
...,...,...
156,2023-09-20,recent indicators suggest economic activity ex...
157,2023-11-01,recent indicators suggest economic activity ex...
158,2023-12-13,recent indicators suggest growth economic acti...
159,2024-01-31,recent indicators suggest economic activity ex...


In [14]:
def construct_labels(fomc, portfolio_size=0.1, holding_periods=[1,5,10]):
    """ 
    Creates labels for a dataset of FOMC statements. Labels are constructed so that they provide
    - stocks in the long
    - stocks in the short
    - which holding period results in the highest annualized return

    :param fomc: contains cleaned FOMC documents and their date of occurence 
    :param portfolio_size: percentage of stocks to be in the long and short
    :param holding_periods: portfolio holding periods to consider
    :
    """
    top_percentile = 1 - portfolio_size
    bottom_percentile = portfolio_size

    fomc_return_data = []

    for index, row in fomc.iterrows():
        fomc_date = row['date']
        statement = row['statement']
        
        # For each holding period, calculate cumulative returns
        for holding_period in holding_periods:
            # Filter returns for T days after the FOMC date
            post_fomc_returns = sp[(sp['date'] > fomc_date) & 
                                                (sp['date'] <= fomc_date + pd.Timedelta(days=holding_period))]
    

            # aggregate returns by stock (permno) to get cumulative returns for T days
            cumulative_returns = post_fomc_returns.groupby('permno')['ret'].sum().reset_index()

            # calculate the to and bottom percentiles for cumulative returns
            long_threshold = cumulative_returns['ret'].quantile(top_percentile)
            short_threshold = cumulative_returns['ret'].quantile(bottom_percentile)

            # select stocks above the top percentile for long, and below bottom percentile for short
            long_stocks = cumulative_returns[cumulative_returns['ret'] >= long_threshold]['permno'].tolist()
            short_stocks = cumulative_returns[cumulative_returns['ret'] <= short_threshold]['permno'].tolist()
            
            
            # store results for each FOMC statement and holding period
            fomc_return_data.append({
                'fomc_date': fomc_date,
                'statement': statement,
                'holding_period': holding_period,
                'long_stocks': long_stocks,
                'short_stocks': short_stocks
            })

    fomc_return_df = pd.DataFrame(fomc_return_data)
    return fomc_return_df

In [15]:
construct_labels(fomc)

,fomc_date,statement,holding_period,long_stocks,short_stocks
0,2000-02-02,immediate release federal open market committe...,1,"[10104, 10693, 10942, 11983, 14323, 14593, 153...","[10138, 10604, 10874, 11703, 12052, 12140, 185..."
1,2000-02-02,immediate release federal open market committe...,5,"[10104, 10942, 14008, 14593, 15579, 18016, 212...","[10138, 10145, 10604, 10786, 10874, 12052, 131..."
2,2000-02-02,immediate release federal open market committe...,10,"[10078, 10104, 10942, 11042, 11976, 12319, 145...","[10145, 10786, 11703, 11955, 12052, 12140, 124..."
3,2000-03-21,immediate release federal open market committe...,1,"[10104, 10890, 11983, 12650, 14277, 14593, 147...","[10145, 10989, 11308, 11404, 12546, 13901, 166..."
4,2000-03-21,immediate release federal open market committe...,5,"[10104, 10107, 10138, 10693, 10874, 12650, 142...","[10786, 10989, 11308, 13100, 14008, 14322, 150..."
...,...,...,...,...,...
478,2024-01-31,recent indicators suggest economic activity ex...,5,[],[]
479,2024-01-31,recent indicators suggest economic activity ex...,10,[],[]
480,2024-03-20,recent indicators suggest economic activity ex...,1,[],[]
481,2024-03-20,recent indicators suggest economic activity ex...,5,[],[]


In [4]:
# train set: 2000-2015
train_df = fomc[(fomc['date'].dt.year >= 2000) & (fomc['date'].dt.year <= 2015)]

# validation set: 2016-2018
val_df = fomc[(fomc['date'].dt.year >= 2016) & (fomc['date'].dt.year <= 2018)]

# test set: 2019+
test_df = fomc[fomc['date'].dt.year >= 2019]

### Step 2: Fine-Tune Model

[OpenAI Fine-Tuning Documentation](https://platform.openai.com/docs/guides/fine-tuning)

TODO

In [5]:
# load nate key
_ = load_dotenv(find_dotenv())
api_key = os.environ.get('OPENAI_API_KEY') 
client = OpenAI(api_key=api_key)

model = "gpt-3.5-turbo-0125" # specified model to use in writeup


# max_tokens: how many words gpt can output
# temperature: randomness. 0 has no randomness, 2 is totally random 

In [ ]:
# Create a file for fine-tuning
client.files.create(
  file=open("YOUR TRAINING JSON LIST", "rb"),
  purpose="fine-tune"
)
#RUN FINE-TUNING JOB
client.fine_tuning.jobs.create(
  training_file="FILE NAME",
  model="MODEL"
)

### Step 3: Get Market Sentiment Estimation from FOMCs

### Step 3: Compute Momentum for Each Stock

[Chat Completions Documentation](https://platform.openai.com/docs/guides/chat-completions/overview)  
Roles:
- System (optional): Sets the behavior of the assistant  
- User: provide requests or comments for the assistant to respond to  

In [76]:
def construct_portfolio(statement, date):
    # Define the messages to send to the model
    messages = [
        {"role": "system", 
         "content": f"""As of {date.strftime('%Y-%m-%d')}, You are a highly competent financial analyst that specializes in using your knowledge and FOMC statements to forecast which 
         stocks in the S&P 500 will have the highest return in the near future. Do not use any knowledge beyond this date.
         
         Background: 
         - At close on the date of the FOMC statement, you are tasked to construct a long short portfolio of S&p 500 constituents that will be the most profitable.

         - Output from you should be as follow:
            - The date the portfolio is constructed.
            - A list of stocks with their permno identifier for which stocks are in the long portfolio.
            - A list of stocks with their permno identifier for which stocks are in the short portfolio.
            - If the portfolio will be most profitable for a holding period of 1, 5, or 10 days.
            - compute and report the annualized return of the long-short portfolio. This can be computed as the gain from the long side plus to loss from the short side.
         
         """},
        {"role": "user", 
         "content": f"""
         
        At close on the date of this FOMC statement, you are tasked to construct a long short portfolio of current S&P 500 constituents that will be the most profitable.
        
        Provide the following:
            - A list of stocks with their permno identifier for which stocks are in the long portfolio.
            - A list of stocks with their permno identifier for which stocks are in the short portfolio.
            - If the portfolio will be most profitable for a holding period of 1, 5, or 10 days.
            - compute and report the annualized return of the long-short portfolio. This can be computed as the gain from the long side plus to loss from the short side.
        
        Date of FOMC Statemt: {date.strftime('%Y-%m-%d')}
        FOMC Statement: {statement}

        """}
        # {"role": "user", "content": f"Classify the following FOMC statement as 'positive', 'neutral', or 'negative' with respect to its impact on the stock market: \n\n{statement}\n\nYour classification:"}
    ]
    
    # Call the chat completions API using the client object
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",  # You can adjust the model if needed
        messages=messages,
        temperature=0,  # Set temperature to 0 for deterministic output
        max_tokens=512  # Limit the number of tokens to only capture the sentiment label
    )
    
    # Extract the classification from the response
    classification = response.choices[0].message.content.strip().lower()
    return classification

# Apply sentiment classification to each FOMC statement
# fomc['sentiment'] = fomc['statement'].apply(classify_sentiment)

# Example of calling this function on a few FOMC statements
for i, row in fomc.head().iterrows():
    result = construct_portfolio(row['statement'], row['date'])
    print(f"Date: {row['date'].strftime('%Y-%m-%d')} \nResult: {result}")
    break


Date: 1999-05-18 
Result: date of portfolio construction: 1999-05-18

stocks in the long portfolio:
1. company a - permno 123456
2. company b - permno 234567
3. company c - permno 345678

stocks in the short portfolio:
1. company x - permno 876543
2. company y - permno 765432
3. company z - permno 654321

most profitable holding period: 5 days

annualized return of the long-short portfolio: 
- assuming the long portfolio gained 3% and the short portfolio lost 2% over the 5-day holding period, the net return would be 1%.
- to annualize this return, we can use the formula: annualized return = ((1 + net return) ^ (365/number of days)) - 1
- annualized return = ((1 + 0.01) ^ (365/5)) - 1
- annualized return = (1.01 ^ 73) - 1
- annualized return = 1.8697 or 186.97% (rounded to two decimal places)

therefore, the annualized return of the long-short portfolio over a 5-day holding period would be approximately 186.97%.


In [25]:
# # Function to classify sentiment using GPT-3.5-turbo
# def classify_sentiment(statement, statement_date):
#     messages = [
#         {"role": "system", "content": f"As of {statement_date.strftime('%Y-%m-%d')}, you are a financial analyst specializing in monetary policy analysis. Do not use any information beyond this date."},
#         {"role": "user", "content": f"Classify the following FOMC statement as 'positive', 'neutral', or 'negative' with respect to its impact on the stock market:\n\n{statement}\n\nYour classification:"}
#     ]
    
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo-0125",
#         messages=messages,
#         temperature=0,
#         max_tokens=3000  # Limit tokens to capture only the sentiment label
#     )
    
#     classification = response.choices[0].message.content.strip().lower()
#     return classification


,permno,date,ret
0,64936,2000-01-03,-0.028662
1,24205,2000-01-03,-0.036496
2,60441,2000-01-03,-0.028926
3,45751,2000-01-03,-0.011757
4,76887,2000-01-03,-0.042553
...,...,...,...
28546,87445,2023-12-29,0.004683
28547,21792,2023-12-29,0.000350
28548,13356,2023-12-29,0.002258
28549,58819,2023-12-29,-0.000390


In [26]:
# Apply sentiment classification to each FOMC statement in the test set
test_df['sentiment'] = test_df.apply(lambda row: classify_sentiment(row['statement'], row['date']), axis=1)

/var/folders/y7/k0h_r9r927j3nghdzjhcr4xh0000gn/T/ipykernel_94076/2569289976.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['sentiment'] = test_df.apply(lambda row: classify_sentiment(row['statement'], row['date']), axis=1)


In [27]:
test_df

,date,statement,sentiment
126,2019-01-30,information received since federal open market...,neutral
127,2019-03-20,information received since federal open market...,neutral
128,2019-05-01,information received since federal open market...,neutral
129,2019-06-19,information received since federal open market...,neutral
130,2019-07-31,information received since federal open market...,neutral
131,2019-09-18,information received since federal open market...,neutral
132,2019-10-30,information received since federal open market...,neutral
133,2019-12-11,information received since federal open market...,neutral
134,2020-01-29,information received since federal open market...,neutral
135,2020-04-29,federal reserve committed using full range too...,neutral


In [28]:
T = 5  # Choose 1, 5, or 10 days


In [29]:
def construct_portfolio(sentiment):
    if sentiment == 'positive':
        action = 'long'
    elif sentiment == 'negative':
        action = 'short'
    else:
        action = 'neutral'
    return action


In [30]:
test_df['position'] = test_df['sentiment'].apply(construct_portfolio)

/var/folders/y7/k0h_r9r927j3nghdzjhcr4xh0000gn/T/ipykernel_94076/3741860187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['position'] = test_df['sentiment'].apply(construct_portfolio)


In [32]:
sp = sp.sort_values(by=['permno', 'date'])

permno             int64
date      datetime64[ns]
ret              float64
dtype: object

In [33]:
def calculate_portfolio_return(position, entry_date, exit_date):
    # Get stock returns between entry and exit dates
    mask = (sp['date'] >= entry_date) & (sp['date'] <= exit_date)
    period_data = sp[mask]
    
    # Assume equal weighting of S&P 500 firms
    daily_returns = period_data.groupby('date')['ret'].mean()
    cumulative_return = (1 + daily_returns).cumprod() - 1
    total_return = cumulative_return.iloc[-1]
    
    if position == 'long':
        portfolio_return = total_return
    elif position == 'short':
        portfolio_return = -total_return
    else:
        portfolio_return = 0.0
    
    # Annualize return
    annualized_return = (1 + portfolio_return) ** (252 / T) - 1
    return annualized_return


In [34]:
portfolio_returns = []

for idx, row in test_df.iterrows():
    statement_date = row['date']
    position = row['position']
    sentiment = row['sentiment']
    
    if position == 'neutral':
        continue  # Skip if neutral
    
    entry_date = statement_date
    exit_date = statement_date + pd.Timedelta(days=T)
    
    annualized_return = calculate_portfolio_return(position, entry_date, exit_date)
    
    portfolio_returns.append({
        'Statement_Date': statement_date,
        'Annualized_Return': annualized_return,
        'Position': position,
        'Sentiment': sentiment
    })


In [35]:
results_df = pd.DataFrame(portfolio_returns)


In [37]:
average_return = results_df['Annualized_Return'].mean()
portfolio_std = results_df['Annualized_Return'].std()

# Assuming risk-free rate is 0
sharpe_ratio = average_return / portfolio_std


KeyError: 'Annualized_Return'

In [14]:
# # Function to classify sentiment using GPT-3.5-turbo (with updated API structure)
# def classify_sentiment(statement):
#     # Define the messages to send to the model
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant that classifies FOMC statements for stock market sentiment analysis."},
#         {"role": "user", "content": f"Classify the following FOMC statement as 'positive', 'neutral', or 'negative' with respect to its impact on the stock market: \n\n{statement}\n\nYour classification:"}
#     ]
    
#     # Call the chat completions API using the client object
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo-0125",  # You can adjust the model if needed
#         messages=messages,
#         temperature=0,  # Set temperature to 0 for deterministic output
#         max_tokens=3000  # Limit the number of tokens to only capture the sentiment label
#     )
    
#     # Extract the classification from the response
#     classification = response.choices[0].message.content.strip().lower()
#     #classification = response.choices[0].message['content'].strip().lower()
#     return classification

# # Apply sentiment classification to each FOMC statement
# fomc['sentiment'] = fomc['statement'].apply(classify_sentiment)

# # Example of calling this function on a few FOMC statements
# for i, row in fomc.head().iterrows():
#     sentiment = classify_sentiment(row['statement'])
#     print(f"Date: {row['date']}, Sentiment: {sentiment}")



Date: 1999-05-18 00:00:00, Sentiment: neutral
0


In [23]:
test_df['position'] = test_df['sentiment'].apply(construct_portfolio)


KeyError: 'sentiment'

In [ ]:
import pandas as pd

# Assume you have already classified the FOMC statements into sentiment (positive, negative, neutral)

# Define function to calculate momentum (previous 1-month return)
def calculate_momentum(stock_data, lookback_days=30):
    # Calculate rolling returns over the lookback period (momentum)
    stock_data['momentum'] = stock_data.groupby('permno')['ret'].rolling(lookback_days).sum().reset_index(level=0, drop=True)
    return stock_data

# Apply momentum calculation to SP500 stock data
sp500_train = calculate_momentum(sp500_train)

# Define a function to construct the long-short portfolio based on sentiment
def construct_portfolio(fomc_date, sentiment, stock_data, lookback_days=30, top_pct=0.1, bottom_pct=0.1):
    # Select stocks' momentum values on the FOMC release date
    stock_momentum = stock_data[stock_data['date'] == fomc_date][['permno', 'momentum']]
    
    # Rank stocks by momentum
    stock_momentum['rank'] = stock_momentum['momentum'].rank(ascending=False)
    
    # Select top and bottom based on sentiment
    num_stocks = len(stock_momentum)
    top_cutoff = int(num_stocks * top_pct)
    bottom_cutoff = int(num_stocks * bottom_pct)
    
    if sentiment == "positive":
        # Long top momentum stocks, short bottom momentum stocks
        long_stocks = stock_momentum.nlargest(top_cutoff, 'momentum')
        short_stocks = stock_momentum.nsmallest(bottom_cutoff, 'momentum')
    elif sentiment == "negative":
        # Short top momentum stocks, long bottom momentum stocks
        long_stocks = stock_momentum.nsmallest(bottom_cutoff, 'momentum')
        short_stocks = stock_momentum.nlargest(top_cutoff, 'momentum')
    
    return long_stocks, short_stocks

# Example: Construct portfolio on an FOMC date with a 'positive' sentiment
fomc_date = pd.Timestamp('2020-01-29')  # Example FOMC date
sentiment = "positive"  # Example sentiment from FOMC statement

# Get long and short portfolios
long_stocks, short_stocks = construct_portfolio(fomc_date, sentiment, sp500_train)
print("Long Stocks:")
print(long_stocks)
print("Short Stocks:")
print(short_stocks)


---

### Fine Tuning

#### Upload Training File

In [ ]:
client = OpenAI()

client.files.create(
  file=open("mydata.jsonl", "rb"),
  purpose="fine-tune"
)

#### Create a Fine-Tuned Model

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-abc123", 
  model="gpt-4o-mini-2024-07-18" # model is the name of the model to finetune
  # validation_file: null, https://platform.openai.com/docs/api-reference/fine-tuning/create
)

#### Using a Fine-Tuned Model

In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-4o-mini:my-org:custom_suffix:id",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)

In [ ]:
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, 
               {"role": "user", "content": "What's the capital of France?"}, 
               {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}